# Vacancy formation energy

Using the iron GAP, calcualte the vacancy formation energy in a 3x3x3 supercell.

In [ ]:
from __future__ import print_function

# stdlib
import os

# non-stdlib
import ase
import numpy as np
from ase.build import bulk
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS
# Use quippy for GAP iron model
from quippy import Potential

In [ ]:
%matplotlib inline

In [ ]:
# primitive cell and bulk with a vacancy
lattice_constant = 2.84

prim = bulk('Fe', a=lattice_constant)
vacancy = (bulk('Fe', a=lattice_constant, cubic=True) * (3, 3, 3))[:-1]

In [ ]:
# Initialise the GAP
cwd = os.getcwd()

try:
    os.chdir('/opt/share/potentials/GAP/Iron/gp33b/')
    fe_gap = Potential("IP GAP", param_filename='gp33b.xml')
finally:
    os.chdir(cwd)

In [ ]:
# Use primitive cell as the zero energy
prim.set_calculator(fe_gap)

ucf = UnitCellFilter(prim)
opt = LBFGS(ucf)
opt.run(fmax=0.0001)

e_prim = prim.get_potential_energy()
v_prim = prim.get_volume()

In [ ]:
# Optimise positions at each volume point
vacancy.set_calculator(fe_gap)

volumes = []
energies = []

for cell_a in np.linspace(vacancy.cell[0][0]*0.98, vacancy.cell[0][0]*1.02, 7):
    vacancy.set_cell((cell_a, cell_a, cell_a), scale_atoms=True)
    opt = LBFGS(vacancy)
    opt.run(fmax=0.001)
    volumes.append(vacancy.get_volume())
    energies.append(vacancy.get_potential_energy())

In [ ]:
from ase.eos import EquationOfState
eos = EquationOfState(volumes, [e - len(vacancy) * e_prim for e in energies])
from ase.units import GPa

v0, e0, B = eos.fit()
print('Bulk modulus:              {:9.3f} GPa'.format(B / GPa))
print('Vacancy formation energy:  {:9.3f} eV'.format(e0))
print('Vacancy volume:            {:9.3f} A³'.format(v0 - 53*v_prim))
eos.plot()